# Основы работы с Git - Разрешение конфликтов

## Мотивация

Когда несколько членов команды работают с одним удаленным репозиторием, то неизбежно будут возникать конфликты при синхронизации изменений.

### Первый разработчик

In [ ]:
cd ~/repo

In [ ]:
rm -rf ~/repo/.ipynb_checkpoints

In [ ]:
echo "Эта строка не вызывает конфликт" >> README.md

In [ ]:
echo "0.2.0" > version.txt

In [ ]:
git commit -am "Добавить коммит для демонстрации конфликта"

In [ ]:
git push origin HEAD:master

### Второй разработчик

In [ ]:
cd ~/local

In [ ]:
git checkout master

In [ ]:
echo "# Version" > version.txt
echo "0.2.1" >> version.txt

In [ ]:
git status

In [ ]:
git commit -am "Изменить версию на 0.2.1"

In [ ]:
commit=$(git log -n 1 --format="%H")

In [ ]:
git fetch origin

In [ ]:
git log -n 1 origin/master

In [ ]:
git rebase origin/master || true

In [ ]:
git status

### Способ 0: Самое простое - это остановить rebase, передав ключ `--abort` команде `git rebase`:

In [ ]:
git rebase --abort

In [ ]:
git status

### Способ 1: Применить изменения из `origin/master`

In [ ]:
git rebase origin/master \
  || true

In [ ]:
git status

In [ ]:
cat version.txt

In [ ]:
git checkout origin/master -- version.txt

In [ ]:
git status

In [ ]:
git rebase --continue

In [ ]:
git status

In [ ]:
git log -n 3

### Способ 2: Применить локальные изменения

In [ ]:
git reset --hard "$commit"

In [ ]:
git log -n 1

In [ ]:
git rebase origin/master \
  || true

In [ ]:
git status

In [ ]:
git checkout master -- version.txt

In [ ]:
git status

In [ ]:
GIT_EDITOR=true git rebase --continue

In [ ]:
git log -n 1

### Способ 3: Ручное решение конфликта

In [ ]:
git reset --hard "$commit"

In [ ]:
git rebase origin/master \
  || true

In [ ]:
git status

In [ ]:
cat version.txt

In [ ]:
echo "#Version" > version.txt
echo "0.2.0" >> version.txt

In [ ]:
git status

In [ ]:
git add version.txt

In [ ]:
git status

In [ ]:
git diff --cached

In [ ]:
GIT_EDITOR=true git rebase --continue

In [ ]:
git log -n 3

In [ ]:
cat version.txt